In [ ]:
from Bio import SeqIO
import pandas as pd
from collections import Counter
from operator import itemgetter

# parse and save record.id of all present viruses
present_viruses = []

for record in SeqIO.parse(str(snakemake.input.viruses), "fasta"):
    present_viruses.append(record.id)

present_viruses_set = set(present_viruses)

# open metadata and save virus-host connections
metadata = pd.read_csv(str(snakemake.input.metadata), sep='\t')
metadata["UViG_split"] = metadata["## UViG"].str.partition("|")[0]
metadata[['superkingdom', 'phylum', 'class', 'order','family', 'genus', 'species']] = metadata['Host taxonomy prediction'].str.split(';', expand=True)
metadata['phylum'] = metadata['superkingdom'].astype(str) + ';' + metadata['phylum'].astype(str)
metadata['class'] = metadata['phylum'].astype(str) + ';' + metadata['class'].astype(str)
metadata['order'] = metadata['class'].astype(str) + ';' + metadata['order'].astype(str)
metadata['family'] = metadata['order'].astype(str) + ';' + metadata['family'].astype(str)
metadata['genus'] = metadata['family'].astype(str) + ';' + metadata["genus"].astype(str)
uvigs = list(metadata["UViG_split"])
uvig_genus = metadata.set_index('UViG_split').to_dict()['genus']
uvig_family = metadata.set_index('UViG_split').to_dict()['family']
uvig_order = metadata.set_index('UViG_split').to_dict()['order']
uvig_class = metadata.set_index('UViG_split').to_dict()['class']
uvig_phylum = metadata.set_index('UViG_split').to_dict()['phylum']
uvig_superkingdom = metadata.set_index('UViG_split').to_dict()['superkingdom']

# define consensus function
def taxa_consensus(node_list, taxa_map):
    taxa = []
    for node in node_list:
        node2 = node.split('|')[0]
        if node2 in taxa_map:
            taxa.append(taxa_map[node2])
    freq = Counter(taxa)
    num_hits = len(taxa)
    if num_hits > 0:
        agreement = max(freq.values())/len(taxa)
        if agreement > snakemake.params.min_agreement:
            consensus = [key for key, value in freq.items() if value == max(freq.values())]
        else:
            consensus = ""
            agreement = 0
    else:
        agreement = 0
        consensus = ""
    return num_hits, agreement, consensus

# iterate through cluster file
votu = []
genus = []
genus_hits = []
genus_agreement = []
family = []
family_hits = []
family_agreement = []
order = []
order_hits = []
order_agreement = []
classes = []
class_hits = []
class_agreement = []
phylum = []
phylum_hits = []
phylum_agreement = []
superkingdom = []
superkingdom_hits = []
superkingdom_agreement = []

# open clustering results
clusters = open(
    str(snakemake.input.clusters) , 'r')


for line in clusters:
    stripped = line.strip()
    centroid, nodes = stripped.split('\t')
    if centroid in present_viruses_set:
        votu.append(centroid.split('|')[0])
        nodes_split = nodes.split(",")
        hits, agreement, consensus = taxa_consensus(nodes_split, uvig_genus)
        genus_hits.append(hits)
        genus_agreement.append(agreement)
        genus.append(consensus)
        hits, agreement, consensus = taxa_consensus(nodes_split, uvig_family)
        family_hits.append(hits)
        family_agreement.append(agreement)
        family.append(consensus)
        hits, agreement, consensus = taxa_consensus(nodes_split, uvig_order)
        order_hits.append(hits)
        order_agreement.append(agreement)
        order.append(consensus)
        hits, agreement, consensus = taxa_consensus(nodes_split, uvig_class)
        class_hits.append(hits)
        class_agreement.append(agreement)
        classes.append(consensus)
        hits, agreement, consensus = taxa_consensus(nodes_split, uvig_phylum)
        phylum_hits.append(hits)
        phylum_agreement.append(agreement)
        phylum.append(consensus)
        hits, agreement, consensus = taxa_consensus(nodes_split, uvig_superkingdom)
        superkingdom_hits.append(hits)
        superkingdom_agreement.append(agreement)
        superkingdom.append(consensus)
    else:
        continue


dict = {'genome':votu,
        'genus':genus, 'genus_hits':genus_hits, 'genus_agreement':genus_agreement,
        'family':family, 'family_hits':family_hits, 'family_agreement':family_agreement,
        'order':order, 'order_hits':order_hits, 'order_agreement':order_agreement,
        'classes':classes, 'class_hits':class_hits, 'class_agreement':class_agreement,
        'phylum':phylum, 'phylum_hits':phylum_hits, 'phylum_agreement':phylum_agreement,
        'superkingdom':superkingdom, 'superkingdom_hits':superkingdom_hits, 'superkingdom_agreement':superkingdom_agreement}

votu_consensus = pd.DataFrame(dict)

votu_consensus = votu_consensus.add_prefix('votu_')
votu_consensus.rename(columns={"votu_genome":"viral_genome"}, inplace=True)
votu_consensus.to_csv(str(snakemake.output.report), index=False)

votu_consensus['votu_consensus_taxonomy'] = ''
votu_consensus['votu_consensus_taxonomy'] = votu_consensus.apply(lambda x: x.votu_superkingdom if x.votu_superkingdom_agreement > snakemake.params.min_agreement else x.votu_consensus_taxonomy, axis=1)
votu_consensus['votu_consensus_taxonomy'] = votu_consensus.apply(lambda x: x.votu_phylum if x.votu_phylum_agreement > snakemake.params.min_agreement else x.votu_consensus_taxonomy, axis=1)
votu_consensus['votu_consensus_taxonomy'] = votu_consensus.apply(lambda x: x.votu_classes if x.votu_class_agreement > snakemake.params.min_agreement else x.votu_consensus_taxonomy, axis=1)
votu_consensus['votu_consensus_taxonomy'] = votu_consensus.apply(lambda x: x.votu_order if x.votu_order_agreement > snakemake.params.min_agreement else x.votu_consensus_taxonomy, axis=1)
votu_consensus['votu_consensus_taxonomy'] = votu_consensus.apply(lambda x: x.votu_family if x.votu_family_agreement > snakemake.params.min_agreement else x.votu_consensus_taxonomy, axis=1)
votu_consensus['votu_consensus_taxonomy'] = votu_consensus.apply(lambda x: x.votu_genus if x.votu_genus_agreement > snakemake.params.min_agreement else x.votu_consensus_taxonomy, axis=1)

votu_consensus[['votu_genome', 'votu_consensus_taxonomy']].to_csv(str(snakemake.output.taxonomy), index=False)
